<br>
<h1 style="color:green;text-align:center;">
    End of Course Assessment (ECA)
</h1>
<p style="text-align:right;">
    Jul 2018
</p>
<hr>
<br>
<br>

<br>
<h2 style="color:blue;text-align:center;">
    Question 1
</h2>
<br>

In [3]:
from abc import ABC , abstractmethod
from datetime import datetime , timedelta

class Item(ABC):
    
    def __init__(self,ID,Item,Quantity,CostPrice):
        
        self._id= ID
        self._item = Item
        self._quantity = Quantity
        self._costPrice = float(CostPrice)
        self._onOffer = False
    
    @property
    def iD(self):
        return self._id
        
    @property
    def quantity(self):
        return self._quantity
    @quantity.setter
    def quantity(self,newValue):
        self._quantity = newValue
    @property
    def costPrice(self):
        return self._costPrice
    @costPrice.setter
    def costPrice(self,newValue):
        self._costPrice = newValue
    
    def onOffer(self):
        if self._onOffer:
            self._onOffer = False
        else:
            self._onOffer = True
            
    def sellingPrice(self):
        return self.profitMargin() + self._costPrice
    
    def offerPrice(self):
        return self.sellingPrice() - self.discount()
    
    @abstractmethod
    def profitMargin(self):
        pass
    @abstractmethod
    def discount(self):
        pass
    def __str__(self):
        
        if self._onOffer:
            status= '**Offer**'
            price = self.offerPrice()
        else:
            status = '(No Offer)'
            price = self.sellingPrice()
            
        return f"{self._id:<6} {self._item:<15} Avaliable = {self._quantity} ${price:0.2f} {status}"
        
class Perishable(Item):
    
    def __init__(self,expiryDate,ID,Item,Quantity,CostPrice):
        self._expiryDate = expiryDate
        super().__init__(ID,Item,Quantity,CostPrice)
    
    def profitMargin(self):
        return 0.25 * self.costPrice
    
    def discount(self):
        
        if (self._expiryDate-datetime.today()).days < 15:
            return 0.30 * self.sellingPrice()
        elif 15 <= (self._expiryDate-datetime.today()).days < 30:
            return 0.20 * self.sellingPrice()
        else:
            return 0.10 * self.sellingPrice()
    
    def __str__(self):
        return f"{super().__str__()} Expiry Date: {self._expiryDate.strftime('%C %b %Y')} Perishable Item"
        
class NonPerishable(Item):
    
    
    def profitMargin(self):
        return 0.30 * self.costPrice
    
    def discount(self):
        return 0.10 * self.sellingPrice()
    def __str__(self):
        return f"{super().__str__()} Non-Perishable Item"
        
class Groceries:
    
    def __init__(self):
        self._collection =[]
    
    def ListItems(self):
        
        s=''
        
        for items in self._collection:
            s+= items.__str__()
        return s
    
    def updateQuantity(self,iD,quantity):
        
        for item in self._collection:
            if item.iD == iD:
                item.quantity = quantity
                return True
        return False

<br>
<h2 style="color:blue;text-align:center;">
    Question 2
</h2>
<br>            

In [7]:
class ModuleScoreException(Exception):
    """
    Module CLass Exceptions
    """

class Module():
    def __init__(self, modCode, modName):
        self._modCode = modCode
        self._modName = modName
        self._minScoresNWeights = {}
    def setMinScoreNWeight(self, componentType, minScore, weight):
        self._minScoresNWeights[componentType] = [minScore, weight]
    def listMinScoresNWeights(self):
        s = 'Minimum Scores for Clearing Components and their weights:\n'
        keys = list(self._minScoresNWeights.keys())
        keys.sort()
        for k in keys:
            s += '{:10}: {:3d} {:3}\n'.format(k, \
            self._minScoresNWeights[k][0],\
            self._minScoresNWeights[k][1])
        return s
    def __str__(self):
        return 'Code: {:5s} Name: {:30s}\n{}'.\
    format(self._modCode, self._modName, self.listMinScoresNWeights())

class ModuleScore():
    
    def __init__(self,module):
        
        self._module = module
        self._moduleScore = {}
    
    def addScore(self,component,score):
        if component not in self._module._minScoresNWeights:
            raise ModuleScoreException("Invalid Component!")
        elif score<0 or score>100:
            raise ModuleScoreException("Invalid Score! Score must be between 0 & 100")
        else:
            self._moduleScore[component] = score
    
    def updateScore(self,component,score):
        self.addScore(component,score)
    
    def removeScore(self,component):
        if component not in self._moduleScore:
            raise ModuleScoreException("Invalid Component!")
        else:
            self._moduleScore.pop(component)
            
    def hasPassedComponenets(self):
        for keys in self._moduleScore:
            if self._moduleScore[keys] < self._module._minScoresNWeights[keys][0]:
                return False
        return True
    
    def getTotalWeight(self):
        summ=0
        for keys in self._module._minScoresNWeights:
            summ+= self._module._minScoresNWeights[keys][1]
        if summ<=0:
            raise ModuleScoreException(f'Total weights of component {summ} is invalid')
        else:
            return summ
            
    def scoreStr(self):
        
        try:
            total_weight = self.getTotalWeight()
        except ModuleScoreException as e:
            return e
        else:     
            keys = list(self._moduleScore.keys())
            keys.sort()
            s='Scores:\n'
            for k in keys:
                k_weight = float(self._module._minScoresNWeights[k][1]) / total_weight
                s+=f"{k:10}: {self._moduleScore[k]:3} @weight {k_weight:0.1f}\n"
            return s
    
    def finalScore(self):
        
        try:
            total_weight = self.getTotalWeight()
        except ModuleScoreException as e:
            return -1
        else:
            finalScore = 0
            
            for keys in self._moduleScore:
                key_weight = self._module._minScoresNWeights[keys][1] / total_weight
                key_weight = round(key_weight,1)
                finalScore+= key_weight * self._moduleScore[keys]
            return round(finalScore)
    
    def grade(self):
        
        if self.finalScore()==-1:
            return 'Invalid Grade'
        else:
            if self.finalScore()>=80:
                return 'A'
            elif 70<= self.finalScore() <80:
                return 'B'
            elif 60<= self.finalScore() <70:
                return 'C'
            elif 50<=self.finalScore()<60:
                return 'D'
            else:
                return 'F'
    
    def __str__(self):
        if self.finalScore()==-1:
            final = 'Invalid Score '
        else:
            final = self.finalScore()
        return f"{self._module.__str__()}\n"\
    f"{self.scoreStr()}Score {final} Grade {self.grade()}"

class Student:
    
    def __init__(self,number,name):
        self._number = number
        self._name = name
        self._moduleScore = {}
    
    def searchModule(self,modCode):
        try:
            return self._moduleScore[modCode]
        except KeyError:
            return None
    
    def add(self,modCode,ModuleScore):
        
        if self.searchModule(modCode):
            return False
        else:
            self._moduleScore[modCode] = ModuleScore 
            return True
    
    def remove(self,modCode):
        try:
            self._moduleScore.pop(modCode)
            return True
        except KeyError:
            return False
    def listModule(self):
        s=''
        for mods in self._moduleScore:
            s+=mods.__str__() + '\n'
        return s
    def __str__(self):
        return f"Student: {self._number} {self._name}\n{self.listModule()}"    

<br>
<h2 style="color:blue;text-align:center;">
    Question 3
</h2>
<br>

In [8]:
from tkinter import *
from tkinter.scrolledtext import ScrolledText
from random import randint

class GuessGame(Frame):
    
    def __init__(self):
        
        Frame.__init__(self)
        self.master.geometry('600x200')
        self.master.title('2 Player Hi-Lo Game')
        self._f1 = Frame(self,width=650,height=100)
        self._f2 = Frame(self,width=650,height=100)
        #------placing frames------
        self._f1.grid(row=0,sticky=W)
        self._f2.grid(row=1,sticky=W)
        #---------Frame 1 Componenets---------------
        self._Player1Label    = Label(self._f1,text='Player 1:')
        self._Player1Entry    = Entry(self._f1)
        
        
        self._Player2Label    = Label(self._f1,text='Player 2:')
        self._Player2Entry    = Entry(self._f1)
        
        self._submittButton   = Button(self._f1,text="Submitt",command=self.submitt)
        self._playAgainButton = Button(self._f1,text='Play Again',command=self.playAgain)
        self._playAgainButton.configure(state=DISABLED)
        #--------------Placing------------------
        self._Player1Label.grid(row=0,column=0,padx=(100,0))
        self._Player1Entry.grid(row=0,column=1)
        self._Player2Label.grid(row=1,column=0,padx=(100,0))
        self._Player2Entry.grid(row=1,column=1,rowspan=1)
        self._submittButton.grid(row=2,column=1,columnspan=2,sticky=W)
        self._playAgainButton.grid(row=2,column=1,pady=7,padx=(55,0))
        #--------Frame2 Componenets
        self._scrlltx = ScrolledText(self._f2,width=72,height=7)
        self._scrlltx.configure(state=DISABLED)
        #-------Placing---------------
        self._scrlltx.pack()
        self._randomNumber = randint(1,100)
        self._play1score = 0
        self._play2score = 0
        self.grid()
        
    def playAgain(self):
        
        self._scrlltx.configure(state=NORMAL)
        
        self._play1score = 0
        self._play2score = 0
        self._randomNumber = randint(1,100)
        
        lis=[self._Player1Entry,self._Player2Entry]
        for x in lis:
            x.delete(0,'end')
        self._scrlltx.delete(1.0,END)
        self._playAgainButton.configure(state=DISABLED)
        self._scrlltx.configure(state=NORMAL)
    
    
    def submitt(self):
        
        play1 = int(self._Player1Entry.get())
        play2 = int(self._Player2Entry.get())
        
        play1status = None
        play2status = None
        self._scrlltx.configure(state=NORMAL)
        
        if play1==self._randomNumber and play2==self._randomNumber:
            self._play1score+=2
            self._play2score+=2
            play1status = "Correct"
            play2status = 'Correct'
            self._playAgainButton.configure(state=NORMAL)
            
        elif play1==self._randomNumber and play2!=self._randomNumber:
            self._play1score+=2
            play1status = 'Correct'
            if play2>self._randomNumber:
                play2status = 'Too High'
            else:
                play2status = 'Too Low'
            self._playAgainButton.configure(state=NORMAL)
        elif play2==self._randomNumber and play1!=self._randomNumber:
            self._play2score+=2
            play2status = "Correct"
            if play1>self._randomNumber:
                play1status = 'Too High'
            else:
                play1status = 'Too Low'
            self._playAgainButton.configure(state=NORMAL)    
            
        elif play1!=self._randomNumber and play2!= self._randomNumber:
            if play1 == play2:
                
                pass
            else:
                if abs(self._randomNumber - play1) < abs(self._randomNumber - play2):
                    self._play1score+=1
                else:
                    self._play2score+=1
                    
            if play1>self._randomNumber:
                play1status="Too High"
            else:
                play1status = 'Too Low'

            if play2>self._randomNumber:
                play2status="Too High"
            else:
                play2status = 'Too Low'
        
        self._scrlltx.insert('end',f"Processing Player 1:  {play1:<3}, Result = {play1status:<8} Score = {self._play1score}\n")
        self._scrlltx.insert('end',f"Processing Player 2:  {play2:<3}, Result = {play2status:<8} Score = {self._play2score}\n")
        self._scrlltx.configure(state=DISABLED)

def main():
    
    game = GuessGame()
    game.mainloop()
    
main()